connection a la base de données 

In [1]:
import mysql.connector
import pandas as pd
# Connexion à la base de données MySQL
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="datascientest",
  password="temp123",
  database="opa"
)

# Créer un curseur pour exécuter les requêtes SQL
mycursor = mydb.cursor()

# Exécuter une requête SQL pour récupérer les données
mycursor.execute("SELECT * FROM opa.historical_klines;")

# Récupérer les résultats
resultats = mycursor.fetchall()

# Fermer la connexion à la base de données
mydb.close()
# Transformer les résultats en DataFrame pandas
df = pd.DataFrame(resultats, columns=["id_symint", "open_time", "open_price", "high_price", "low_price", 
                                      "close_price", "volume", "close_time", "quote_asset_volume", 
                                      "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume"])

# Exporter le DataFrame dans un fichier CSV
df.to_csv("donnes binance historiques.csv", index=False)



lecture du fichier dans un dataframe df

In [2]:
df=pd.read_csv('donnes binance historiques.csv')
df.head()

,id_symint,open_time,open_price,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,1,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181000,1502945999999,202366.0,171.0,35.160500,150952.00
1,1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234900,1502949599999,100305.0,102.0,21.448100,92608.30
2,1,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229690,1502953199999,31282.3,36.0,4.802860,20795.30
3,1,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443250,1502956799999,19241.1,25.0,2.602290,11291.30
4,1,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807,1502960399999,4239.5,28.0,0.814655,3552.75


importation des bibliotheques

In [3]:
import pandas as pd
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split

conversion a les colonnes 'open_time' et 'close_time' au format datetime garce a la methode to_datetime de Pandas

In [4]:
# Conversion des timestamps en date
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
df.head()

,id_symint,open_time,open_price,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,1,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181000,2017-08-17 04:59:59.999,202366.0,171.0,35.160500,150952.00
1,1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234900,2017-08-17 05:59:59.999,100305.0,102.0,21.448100,92608.30
2,1,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229690,2017-08-17 06:59:59.999,31282.3,36.0,4.802860,20795.30
3,1,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443250,2017-08-17 07:59:59.999,19241.1,25.0,2.602290,11291.30
4,1,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,2017-08-17 08:59:59.999,4239.5,28.0,0.814655,3552.75


In [5]:
# Renommer les colonnes pour une meilleure compréhension
df.rename(columns={'open_price': 'prix-ouverture', 'close_price': 'prix_fermeture','number_of_trades':'Nb_trades'},
          inplace=True)

# Supprimer les colonnes inutiles
df.drop(columns=['high_price', 'low_price', 'quote_asset_volume',
                 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume'], 
        inplace=True)
print(df.columns)

Index(['id_symint', 'open_time', 'prix-ouverture', 'prix_fermeture', 'volume',
       'close_time', 'Nb_trades'],
      dtype='object')


In [7]:
# Création de la variable cible
df['target'] = (df['prix_fermeture'].shift(-1) > df['prix_fermeture']).astype(int)

# Suppression de la dernière ligne pour éviter les valeurs nulles
df.drop(df.tail(1).index, inplace=True)

# Définition des features et de la variable cible
X = df[['prix-ouverture', 'prix_fermeture', 'volume']]
y = df['target']

# Séparation des données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# Entraînement du modèle RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
rfc.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = rfc.predict(X_test)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
print('Accuracy: {:.2f}'.format(accuracy))
print('Recall: {:.2f}'.format(recall))
print('ROC AUC: {:.2f}'.format(roc_auc))

Accuracy: 0.52
Recall: 0.96
ROC AUC: 0.50


In [11]:
# Prédiction d'achat ou de vente
current_data = [[df['prix-ouverture'].iloc[-1], df['prix_fermeture'].iloc[-1], df['volume'].iloc[-1]]]
prediction = rfc.predict(current_data)[0]

if prediction == 1:
    print('Il est recommandé d\'acheter.')
else:
    print('Il n\'est pas recommandé d\'acheter.')

# Stratégie de vente
profit_threshold = 1.05 # Seuil de profit à 5%
loss_threshold = 0.95 # Seuil de perte à 5%

Il est recommandé d'acheter.


C:\Users\Souhi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
